In [3]:
import numpy
import itertools
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from scipy import stats
from Bio.Alphabet import IUPAC
#from itertools import permutations

AA=["A","C","D","E","F","G","H","I","K","L","M","N","P","Q","R","S","T","V","W","Y"]
SC=["1","2","3","4","5","6","7"]


### Side chain
original  
ACDEFGHIKLMNPQRSTVWY  
12445131311222322155




In [4]:
dipep = [''.join(i) for i in itertools.product(AA, repeat = 2)]
di_sc = [''.join(i) for i in itertools.product(SC, repeat = 2)]
print(di_sc)
f_num=56

['11', '12', '13', '14', '15', '16', '17', '21', '22', '23', '24', '25', '26', '27', '31', '32', '33', '34', '35', '36', '37', '41', '42', '43', '44', '45', '46', '47', '51', '52', '53', '54', '55', '56', '57', '61', '62', '63', '64', '65', '66', '67', '71', '72', '73', '74', '75', '76', '77']


In [5]:
myseq="AILMVNQSTGPCHKPDEFWY"
trantab2=myseq.maketrans("AILMVNQSTGPCHKPDEFWY","11111222233455566777")
arr = numpy.empty((0,f_num), dtype=numpy.float)
class_arr = numpy.empty((0), dtype=numpy.int)
#print(arr.shape)
#print(arr)
all_fasta=("../major_capsid.fasta","../minor_capsid.fasta","../baseplate.fasta",
          "../major_tail.fasta","../minor_tail.fasta","../portal.fasta",
          "../tail_fiber.fasta","../tail_sheath.fasta","../collar.fasta",
          "../htj.fasta")
#print(all_fasta)
prot_class=0;
for file in all_fasta:
    print('####################' + file)
    for record in SeqIO.parse(file, "fasta"):
        #print(record.id)
        ll=len(record.seq)
        #xx=Seq(str(record.seq),IUPAC.protein)
        X = ProteinAnalysis(record.seq.__str__().replace('X','A').replace('J','L'))
        #X = ProteinAnalysis(str(xx))
        tt= [X.isoelectric_point(), X.instability_index(),ll,X.aromaticity(),
            X.molar_extinction_coefficient()[0],X.molar_extinction_coefficient()[1],
            X.gravy()]
        tt_n = numpy.asarray(tt,dtype=numpy.float)
        myseq=str(record.seq).translate(trantab2)
        dipep_count=[myseq.count(i)/ll for i in di_sc]
        dipep_count_n = numpy.asarray(dipep_count,dtype=numpy.float)
    
        cat_n= numpy.append(dipep_count_n,tt_n)
        cat_n = cat_n.reshape((1,cat_n.shape[0]))

        arr = numpy.append(arr,cat_n , axis=0)
        class_arr = numpy.append(class_arr,prot_class)
    prot_class+=1
    

    

####################../major_capsid.fasta
####################../minor_capsid.fasta
####################../baseplate.fasta
####################../major_tail.fasta
####################../minor_tail.fasta
####################../portal.fasta
####################../tail_fiber.fasta
####################../tail_sheath.fasta
####################../collar.fasta
####################../htj.fasta


In [6]:
 print(arr.shape)

#arr_z = numpy.apply_over_axis(stats.zscore,arr,0)
arr_z=numpy.apply_along_axis(stats.zscore,0,arr)
#arr_z=numpy.apply_along_axis(stats.zscore,0,arr)
print(arr_z[:10,])
#print(arr)
#print(arr_z.shape)
#print(prot_class)

(25675, 56)
[[ 1.80452093e+00  5.09581312e-01  1.46100281e+00 -5.34959735e-01
   1.85424548e-01 -5.15846948e-01 -4.51878866e-01  3.37067861e-01
  -1.03000215e+00 -5.96530762e-01 -5.75704636e-01 -1.10142275e-01
   6.89190427e-01 -1.98991324e-01 -6.88073250e-02  1.11654957e+00
  -4.50432001e-01  2.07995874e+00 -5.66997329e-01 -5.30850203e-01
   6.79318129e-01  3.65544752e-01 -5.72098403e-01 -3.35929286e-01
  -1.29578548e-01 -4.07617335e-01 -4.34972787e-01 -4.01342631e-01
   2.88710114e-02 -6.45642260e-01  2.56114469e-01 -3.88508348e-01
   2.24257112e-01 -1.51349638e-01 -8.56366397e-01  6.00695690e-01
   6.53567835e-02 -5.31904197e-01 -4.22103245e-01 -8.56900663e-01
   3.16575990e-01 -1.09667398e+00  1.98550400e-01 -9.26137849e-01
   1.54649525e-01 -3.94911844e-01 -4.00430928e-01 -1.10451158e+00
  -5.56905727e-01 -9.13714260e-01 -3.99215599e-01 -2.67864995e-01
  -8.59932516e-01 -4.36528395e-01 -4.39185078e-01  1.82675353e+00]
 [ 5.97673030e-01  6.13420227e-01 -1.68728548e-01  7.25153954e-

In [7]:
mean_arr=numpy.apply_along_axis(numpy.mean,0,arr)
#print(mean_arr)
print(mean_arr.shape)
std_arr=numpy.apply_along_axis(numpy.std,0,arr)
print(std_arr.shape)

(56,)
(56,)


In [8]:
#targets = numpy.array([[2, 3, 4, 0]]).reshape(-1)

nb_classes = 10
one_hot_targets = numpy.eye(nb_classes)[class_arr]
print(one_hot_targets)
print(one_hot_targets.shape)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
(25675, 10)


In [9]:

final = numpy.concatenate((arr_z, one_hot_targets), axis=1)
final_raw = numpy.concatenate((arr, one_hot_targets), axis=1)

In [ ]:
#numpy.savetxt('test.out', final, delimiter='\t') 
#numpy.savetxt('test2.out', final_raw, delimiter='\t') 

In [10]:
numpy.random.shuffle(final)
numpy.random.shuffle(final_raw)

In [11]:
import pickle
##number of proteins for training
tt=25000
##number of features

pickle.dump(final[0:tt,0:f_num], open( "di_sc_train_x.p", "wb" ) )
pickle.dump(final[tt:,0:f_num],  open( "di_sc_test_x.p" , "wb" ) )
pickle.dump(final[0:tt,f_num:], open( "di_sc_train_y.p", "wb" ) )
pickle.dump(final[tt:,f_num:],  open( "di_sc_test_y.p" , "wb" ) )
pickle.dump(mean_arr,  open( "di_sc_mean.p" , "wb" ) )
pickle.dump(std_arr,  open( "di_sc_std.p" , "wb" ) )

In [ ]:
#pickle.dump(final_raw[0:tt,0:f_num], open( "dipep_raw_new_train_x.p", "wb" ) )
#pickle.dump(final_raw[tt:,0:f_num],  open( "dipep_raw_new_test_x.p" , "wb" ) )
#pickle.dump(final_raw[0:tt,f_num:], open( "dipep_raw_new_train_y.p", "wb" ) )
#pickle.dump(final_raw[tt:,f_num:],  open( "dipep_raw_new_test_y.p" , "wb" ) )